In [1]:
from abides_markets.configs import rmsc04
from abides_core import abides
from abides_markets.utils import generate_latency_model
from abides_core.utils import get_wake_time, str_to_ns
from abides_core.network import BANetwork, CentralizedNetwork
from abides_markets.agents import InsiderValueAgent
from abides_markets.agents.network_agents import FollowerValueAgent, FollowerNoiseAgent
from itertools import count
from abides_markets.configs import communication

In [2]:
rmsc04_based_config = rmsc04.build_config()
base_config_agents_sum = len(rmsc04_based_config['agents'])
id_generator = count(1)

In [3]:
# najpierw generujemy centralized network: zakładam triady: powiedzmy na razie 1 insider skomunikowany z dwoma followerami 
insider_0 = InsiderValueAgent(id=next(id_generator),lambda_a=5.7e-12,horizon=str_to_ns("1h"))
follower_0 = FollowerValueAgent(id = next(id_generator),lambda_a=5.7e-12)
market_open = rmsc04_based_config['start_time'] + str_to_ns("11:30:00")
market_close = rmsc04_based_config['stop_time'] - str_to_ns("1s")
follower_1 = FollowerNoiseAgent(id = next(id_generator),wakeup_time = market_open)

In [4]:
starting_graph = CentralizedNetwork.construct_from_agent_list(central_agent=insider_0, agent_list=[follower_0, follower_1])

In [5]:
# generujemy pozostałych agentów, powiedzmy analogicznie 
all_follower_value_agents = [FollowerValueAgent(id = next(id_generator),lambda_a=5.7e-12) for i in range(50)]
all_follower_noise_agents =  [FollowerNoiseAgent(id = next(id_generator),wakeup_time = market_open) for i in range(500)]
agents = all_follower_value_agents
agents.extend(all_follower_noise_agents)

In [6]:
# generujemy graf 
complete_graph = BANetwork.construct_from_agent_list(starting_state=starting_graph, new_agents=agents, m= 2)
new_agents = sorted(complete_graph.agents, key = lambda agent: agent.id)

In [7]:
new_config = communication.build_config(complete_graph)
end_state_2 = abides.run(config=new_config)

[5572] INFO abides Simulation Start Time: 2023-06-12 18:18:28.665232
[5572] INFO abides_core.kernel --- Simulation time: 2021-02-05 00:00:00, messages processed: 0, wallclock elapsed: 0.00s ---
[5572] INFO abides_core.kernel --- Simulation time: 2021-02-05 09:53:28, messages processed: 100,000, wallclock elapsed: 2.98s ---
[5572] INFO abides_core.kernel --- Simulation time: 2021-02-05 10:30:21, messages processed: 200,000, wallclock elapsed: 6.98s ---
[5572] INFO abides_core.kernel --- Simulation time: 2021-02-05 11:15:24, messages processed: 300,000, wallclock elapsed: 11.73s ---
[5572] INFO abides_core.kernel --- Simulation time: 2021-02-05 11:57:42, messages processed: 400,000, wallclock elapsed: 16.82s ---
[5572] INFO abides_core.kernel --- Simulation time: 2021-02-05 12:37:23, messages processed: 500,000, wallclock elapsed: 21.90s ---
[5572] INFO abides_core.kernel --- Simulation time: 2021-02-05 13:12:58, messages processed: 600,000, wallclock elapsed: 26.57s ---
[5572] INFO abid

In [8]:
sorted([agent.id for agent in new_config['agents']])

[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 161,
 162,
 163,
 164,
 165,
 166,
 167,
 168,
 169,
 170,
 171,
 172,
 173,
 174,
 175,
 176,
 177,
 178,
 179,
 180,
 181,
 182,
 183,
 184,


In [9]:
end_state_2['agents']

 ...]

In [10]:
# cena 

end_state_2['agents'][0].order_books['ABM']

In [11]:
from abides_core.utils import parse_logs_df, ns_date, str_to_ns, fmt_ts
logs_df = parse_logs_df(end_state_2)

In [12]:
logs_df.loc[logs_df['agent_type']=='ExchangeAgent'][2000:2050]

,EventTime,EventType,ScalarEventValue,agent_id,agent_type,EmptyEvent,CASH,ABM,time_placed,symbol,quantity,side,order_id,fill_price,tag,limit_price,is_hidden,is_price_to_comply,insert_by_id,is_post_only
2000,1612483200006609090,MarketHoursRequestMsg,MarketHoursRequestMsg(message_id=1244),0,ExchangeAgent,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2001,1612483200006612261,MarketClosePriceRequestMsg,MarketClosePriceRequestMsg(message_id=2873),0,ExchangeAgent,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2002,1612483200006612261,MarketHoursRequestMsg,MarketHoursRequestMsg(message_id=2874),0,ExchangeAgent,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2003,1612483200006631320,MarketClosePriceRequestMsg,MarketClosePriceRequestMsg(message_id=2333),0,ExchangeAgent,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2004,1612483200006631320,MarketHoursRequestMsg,MarketHoursRequestMsg(message_id=2334),0,ExchangeAgent,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2005,1612483200006636400,MarketClosePriceRequestMsg,MarketClosePriceRequestMsg(message_id=2823),0,ExchangeAgent,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2006,1612483200006636400,MarketHoursRequestMsg,MarketHoursRequestMsg(message_id=2824),0,ExchangeAgent,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2007,1612483200006659507,MarketClosePriceRequestMsg,MarketClosePriceRequestMsg(message_id=3227),0,ExchangeAgent,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2008,1612483200006659507,MarketHoursRequestMsg,MarketHoursRequestMsg(message_id=3228),0,ExchangeAgent,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2009,1612483200006663705,MarketClosePriceRequestMsg,MarketClosePriceRequestMsg(message_id=1181),0,ExchangeAgent,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
logs_df['EventType'].unique()

array(['AGENT_TYPE', 'MarketClosePriceRequestMsg',
       'MarketHoursRequestMsg', 'BookImbalanceSubReqMsg',
       'QuerySpreadMsg', 'BEST_BID', 'BEST_ASK', 'QueryTransactedVolMsg',
       'LAST_TRADE', 'STARTING_CASH', 'HOLDINGS_UPDATED', 'BID_DEPTH',
       'ASK_DEPTH', 'IMBALANCE', 'FINAL_HOLDINGS', 'FINAL_CASH_POSITION',
       'MARK_TO_MARKET', 'MARKED_TO_MARKET', 'ENDING_CASH',
       'FINAL_VALUATION', 'ORDER_SUBMITTED', 'ORDER_EXECUTED',
       'ORDER_ACCEPTED', 'CANCEL_SUBMITTED', 'ORDER_CANCELLED'],
      dtype=object)

In [15]:
g = complete_graph.generate_networkx_object()

AttributeError: 'BANetwork' object has no attribute 'generate_networkx_object'

In [ ]:
dir(complete_graph)